# Exploring conformational space of selected macrocycles - "M1", Part 2

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Tue Apr 11 08:19:52 2017


In [62]:
# Functions used in this notebook:
def grep_energies_from_csvfile(fcsv):
    energies = {}
    with open(fcsv,'r') as f:
        lines = f.readlines()
        for line in lines:
            words = line.split()
            name = words[0].rsplit("_", 5)[0]
            energies[name] = float(words[1])
    return energies

def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

def get_moldict_from_inplist(inplist):
    moldict = {}
    for i, inp in enumerate(inplist):
        mol = Chem.MolFromMolFile(inp)
        name = os.path.basename(os.path.splitext(inp)[0]).rsplit("_", 5)[0]
        model = "_".join(os.path.basename(os.path.splitext(inp)[0]).rsplit("_", 2)[1:])
        moldict[name] = mol
    return moldict

def check_freq(freq_inp_list):
    negative_freq_dict = {}
    for inp in freq_inp_list:
        count = 0
        with open(inp, "r") as f:
            lines = f.readlines()
            for line in lines:
                freq = line.split().strip()[1]
                print(freq)
                if float(freq) < 0.0:
                    count += 1
            if count > 0:
                negative_freq_list[os.path.basename(os.path.splitext(inp)[0]).rsplit("_", 5)[0]] = count
    return negative_freq_dict

In [53]:
# decide what is the "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures
smiles = 'O=C1NCCNC(=O)c2nc(C(=O)NCCNC(=O)c3nc1ccc3)ccc2'
core_smiles = 'n1ccccc1'

m1 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m1 = m1.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m1 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m1/balloon/m1_crystal.sdf')
m1_crystal = templ_m1[0]

## Geometry optimization in Gaussian 09 software

### Isolated "M1" molecule in the gas phase - computational setup

We will use few models to determine the geometry and energy of M1 conformers. All calculations in Gaussian 09.

| Abbrev. | XC Model | Basis set | "Geometry" keywords | "Freq" keywords |
|---------|----------|-----------|---------------------|-----------------|
| s1 | PBE | 6-31G(d) |`#p PBEh1PBE/6-31G(d') opt test nosymm pop=full`|`#p PBEh1PBE/6-31G(d') freq test geom=check guess=read`|
| s2 | B97D | 6-31G(d) | `#p B97D/6-31G(d') opt test nosymm pop=full` | `#p B97D/6-31G(d') freq test geom=check guess=read` |
| s3 | B97D | cc-pVTZ |                     |                 |




### Isolated "M1" molecule in the gas phase - results

In [57]:
inps_m1_gaussian_s1 = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/*pbe_631gd.sdf')
inps_m1_gaussian_s2 = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/*b97d_631gd.sdf')

mol_m1_gaussian_s1 = get_moldict_from_inplist(inps_m1_gaussian_s1)
mol_m1_gaussian_s2 = get_moldict_from_inplist(inps_m1_gaussian_s2)

for key, mol in mol_m1_gaussian_s1.items():    
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in mol_m1_gaussian_s1.items(): 
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()    


We will search for potential duplicates (based on RMSD and on energy) and remove them:

In [63]:
f_s1 = "/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/energy_pbe_631gd.csv"
e_m1_gaussian_s1 = grep_energies_from_csvfile(f_s1)
   
allmol_m1_s1 = {}
allmol_m1_s1.update(mol_m1_gaussian_s1)

energy_m1_s1 = {}
energy_m1_s1.update(e_m1_gaussian_s1)

rms_m1_s1 = {}
for key, mol in allmol_m1_s1.items():
    rms_m1_s1[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m1_crystal))

In [60]:
rms_thresh = 0.05

rms_sorted_s1 = sorted(rms_m1_s1.items(), key=lambda x: x[1])


print("List sorted by RMS:")
for i, t in enumerate(rms_sorted_s1):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted_s1[i][0], energy_m1_s1[rms_sorted[i][0]], rms_sorted_s1[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted_s1 = find_duplicates(rms_sorted_s1, energy_m1_s1, rms_thresh)

for mol in to_be_deleted_s1:
    del allmol_m1_s1[mol]
    del energy_m1_s1[mol]
    del rms_m1_s1[mol]      

List sorted by RMS:
name = m1_rdkit_smi_10, E = -1324.876446, RMS = 0.934897
name = m1_rdkit_smi_51, E = -1324.871111, RMS = 0.967572
name = m1_b_sdf_3, E = -1324.915715, RMS = 1.216777
name = m1_b_smi_3, E = -1324.915584, RMS = 1.217223
name = m1_b_smi_0, E = -1324.913243, RMS = 1.242958
name = m1_b_sdf_0, E = -1324.913314, RMS = 1.244748
name = m1_rdkit_smi_12, E = -1324.873430, RMS = 1.428213
name = m1_rdkit_smi_20, E = -1324.873449, RMS = 1.435037
name = m1_rdkit_smi_16, E = -1324.869304, RMS = 1.543641
name = m1_rdkit_smi_32, E = -1324.861975, RMS = 1.854721
name = m1_rdkit_sdf_26, E = -1324.861785, RMS = 1.902581
Conformers which will be deleted:
['m1_rdkit_smi_51', 'm1_b_smi_3', 'm1_b_smi_0', 'm1_b_sdf_0', 'm1_rdkit_smi_12', 'm1_rdkit_sdf_26']


We can now present the  remaining conformers:

In [64]:
print(len(allmol_m1_s1))

for key, mol in allmol_m1_s1.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m1_crystal,atomMap=list(zip(core_mol,core_m1)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m1_s1.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

11


In [ ]:
# We have also computed harmonic frequencies. We can check whether all saved conformers are local minima:

freq_m1_gaussian_s1 = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m1/gaussian_results/freq*pbe_631gd.csv')
check_freq(freq_m1_gaussian_s1)
